# L6118 - Simple Steps to Speed Up Your C Code Dramatically with GPU

Abstract:
Many scientists have developed their own old trusted C code, and they would like to make it run faster using the new GPU technology. However, they don't how to start this process. In this tutorial, we provide simple steps to speed up a piece of C code. To do so, we have selected the most common algorithms where the parallel architecture benefits are best and we describe the little details encountered in the transition from C to CUDA programming. 

This tutorial will cover how to do:
    * Allocate Dynamic memory in the device 
    * Call functions 
    * Data transfer from device to host and host to device
    * Execution of kernels
    * Implementation of averages and histograms
    
We organize the tutorial in the following manner:
    * Review of CUDA architecture and parallel programming.
    * Reduce and Scan Algorithms
    * Examples and Exercises:

# Introduction

One of the key components in migrating from C to CUDA code (for a beginner) is to identify what implementation can be executed easily and will reduce the computation time significantly. To this aim,  it is necessary to learn the basics of CUDA architecture and a set of parallel algorithms which are frequently used and relatively easy to implement. This will help to decrease the slope of our learning curve and gradually deepen in the CUDA parallel world.

In the serial world of programming, we love loops because they allow us to solve problems by performing operations sequencially. In addition, they are easy to understand, implement, and debug. However, performing sequential  operations kills our performance when the size of the data to analyze and/or operations within the loops increases.
In short, parallel coding represents "breaking inefficient loops". Here, we present two algorithms REDUCE and SCAN that allow us to break loops and exploit the parallel power of our GPUs, but first let's summarize how to call C functions, CUDA kernels, allocating device memory, and launching kernels.



# BASICS:

First things first, the main function has to be in a .cu file, and to be able to call CUDA functions, we need to add the include for the CUDA header file:


### Declaration of functions

In order to call a "C" function in the CUDA main function that does not use any CUDA class or function and it is located in other C file, it has to be declared as,

In order to call a user made CUDA function (a function that uses any CUDA class) and that it is not a kernel and it has to be located in other *.cu file, it has to be declared like, 

In order to call a Kernel that is outside of the main.cu file, it has to be declared as,

### GPU information

To make your code suitable for different GPU, it is important to code base on the characteristics of any GPU. We can obtain this information by using the CUDA class __cudaDeviceProp__ and CUDA functions. Among them, we have 
- __cudaGetDeviceCount__ retuns the number of GPUs in your node or machine.
- __cudaGetDevice(&dev)___ returns the device on which the active host thread executes the device code in dev.
- __cudaGetDeviceProperties(&deviceName,dev)__ returns in *deviceName the properties of device dev.

In addition, the __cudaDeviceProp__ class has many member elements such as:
- __maxThreadsPerBlock__ gives the maximum number of threads per block that your GPU can handle.
- __totalGlobalMem__ gives the total size of Global memory.
- __sharedMemPerBlock__ gives the total size of Shared memory in each block.
  
among other features. We have extracted the main properties in a CUDA function __"void printDevProp(cudaDeviceProp devProp)"__, a more extended version of this function can be obtain in ~/NVIDIA_CUDA-7.5_Samples/1_Utilities/deviceQuery/deviceQuery.cpp. We also provided an small program cuda-features that runs this function.  


Now, let's test what we have learn.

## Exercise 1:

All files are in  basics/exercise1

In [ ]:
!ls basics/exercise1

In [ ]:
!more basics/exercise1/kernels.cu

=#include <stdio.h>
#include <math.h>


void printDevProp(cudaDeviceProp devProp)
{
    printf("\n\nGPU features:");
    printf("\n\tMajor revision number:         %d\n",  devProp.major);
    printf("\tMinor revision number:         %d\n",  devProp.minor);
    printf("\tName:                          %s\n",  devProp.name);
    printf("\tTotal global memory:           %lu kB\tTotal %lu Kilo floats\n",  devProp.totalGlobalMem/1024,devProp.totalGlobalMem/(sizeof(float)*1000));
    printf("\tTotal shared memory per block: %lu kB\t\tTotal %lu Kilo floats\n",  devProp.sharedMemPerBlock/1024,devProp.sharedMemPerBlock/(sizeof(float)*1000));
    printf("\tTotal registers per block:     %d\n",  devProp.regsPerBlock);
    printf("\tWarp size:                     %d\n",  devProp.warpSize);
    printf("\tMaximum memory pitch:          %lu\n",  devProp.memPitch);
    printf("\tMaximum threads per block:     %d\n",  devProp.maxThreadsPerBlock);
    for (int i = 0; i < 3; ++i)
        printf("\tMaxim

In [2]:
!more basics/exercise1/main.cu

=/* program that prints the GPU features*/ 

#include <stdio.h>


int main(int argc, char *argv[])
{

  cudaDeviceProp prop;
  int device;

  cudaGetDevice(&device);
  cudaGetDeviceProperties (&prop,device);
  printDevProp(prop); 


  return(0); 
}

>

After, you corrected the code. Compile it by using nvcc main.cu kernels.cu -o main 

In [ ]:
!ls

## Lunching Kernels and Device memory Allocation

Now, that we know how to compile and call functions, let's learn how to luch kernels.

__Kernels__ are sequences of instructions that are going to be processed by each thread. They are written similarly to C functions, but they usually have a prefix  

and they are called or "launched" in the following manner,

__Blocks__ allow us to identify a set of threads that are going to be processed by a single SM, and thus they have access to the same share memory. The number of blocks and number of threads can be tailor to the design of your program, but their maximum value is specific of each GPU. In the previous exercise, we show how to obtain parameters like the maximum number of threads per blocks, the maximum number of threads in a particular dimension, among others. The blocks can be arranged in arrays of 1, 2 ,3 dimensions, the specifications of the arrangement of blocks is called __Grid dimensionality__.

__Threads__ are our engines to perform operations through kernels. They have their unique Id, base on the number of threads per block and the number of blocks. They also can be 1D, 2D, or 3D, and for these examples, we are going to use only 1D arrangement of threads per block.

__Input parameters__  If they are arrays, they have to be allocated specifically for the GPU/device. Since most of the programs allocate memory dynamically, we are going to focus on such. One main difference in CUDA programming is device's arrays can not be accessed by the host or CPU. In other to have access, we have to transfer memory from device to host and vice-versa. CUDA has specific functions to do that such as,   

Note that both arrays have to have the same size. 
To allocate memory for the device, we use __cudaMalloc__, which requires to specify the size of the array and the type like,

where, __d_array__ is the name of the array and  __size_array__ is the size of array.

After this summary of CUDA programming, let's launch some kernels!

## Exercise 2:

__A.__ Generate an array of 10000 numbers, with values increasing from 1 to 10000. For this part of the exercise, we provide the kernel, but you have to add the missing parts in the main.

In [6]:
 tail -14 basics/exercise2/partA/kernerls.cu

__global__ void int_generator (int *d_out,unsigned long int size){

    int index_x = blockIdx.x * blockDim.x + threadIdx.x;
    int index_y = blockIdx.y * blockDim.y + threadIdx.y;
    int grid_width = gridDim.x * blockDim.x;

    /* map the two 2D indices to a single linear, 1D index */

    int gId  = index_y * grid_width + index_x;
    if(gId<size){
      d_out[gId]=gId;
    }     

}


__B__. Using the example in part A, generate 100 arrays, where each array will have increasing values from 1 to 1000.
The C and CUDA templates for this exercise are: 

Now that we know how to lunch kernel, we are ready to learn basic parallel algorithms.

# REDUCE: 

Reduce as its name suggests is a primitive or function that reduces many inputs to one or more outputs by performing an associative operation on the inputs. __Average__ is an ideal case where it can be used. Let's go further on the details of its implementation. 

The serial version of computing the average is,

The problem of implementing this algorithm is that each operation depends on a previous one. So, if we want to implement this approach in a GPU, each thread would have to access the result of the previous one, thus they can be computed at the same time, which is not good.

We need to find the way of obtaining the same output, by distributing operations that do not depend on the results of others and do not need to have access to the same memory location, so that they can be computed in parallel. 

To break this for loop into independent operations, the previous scheme won't help much; however, if we expand the array in a different manner, we can express the same sum by accessing different locations at once. For example, if __a__ is an array of size of 8, its average is



If we associate them and accumulate its values at different steps, such as

__a[0]__ will have all the terms of the array. The previous arrangement can be represented in C code as,

Let's take a look in CUDA.

## Example1: Computing the average of a set of numbers 

For this exercise, we are going to use the program that we implemented in the last exercise, but in addition, we are going to compute the average of the 10000 numbers generated. The kernel that is going to performed the average is called __reduce__,

In [13]:
head -44 reduce/example1/kernerls.cu |tail -30 

__global__ void reduce (float *d_in,float *d_out,unsigned long int size){

    extern __shared__ float sdata[];

   
    // map the two 2D indices to a single linear, 1D index
    int gId  = blockIdx.x*blockDim.x +threadIdx.x;
    int lId  = threadIdx.x;
    int bId  = blockIdx.y * gridDim.x + blockIdx.x;

   if(gId<size)
    sdata[lId]=d_in[gId];
   else
    sdata[lId]=0.0;

    __syncthreads();    

   for(unsigned int s =blockDim.x/2;s>0;s >>=1){

        if(lId<s){
	 sdata[lId]+=sdata[lId+s];
        }
    __syncthreads();

   }
    if(lId==0){
      d_out[bId]=sdata[0];
    }     

}


There, we have two new expressions, __extern__  ___shared___ __ __ and ____syncthreads()__.  

__extern__ ___shared___ is used to declare the share memory that is going to be used in the Kernel. We could use Global memory instead, but the access to the share memory is faster than to the global memory. So, it will speed up our code. 

____syncthreads()__ is a command that holds the execution of the kernel until all threads of the block reach that point. We use it here to __wait until__ all the threads copy the values of the array from global memory to share memory. To then perform the for-loop.

## Exercise1. Computing the average visited sites of a Random Walk

Compute numerically the time dependence of the average visitted sites of a random walk in a 3D lattice $\langle W\rangle$. 

There are many ways of solving this problem, however we are going to solve by simply marking and counting the places where the random walk has jumped in time and average them. Although the concept is very simple, it will require to add a 3D matrix at every time step $t$. Thus, the time increases, the computational time is increases approximately $t*N_{x}N_{y}N_{z}$, where $N_i$ is the maximum distance in the x,y, or z direction. 

The files for this exercise is located in reduce/exercise1


In [ ]:
To aid its implementation, we provide the C code in main.c 

In [14]:
head -75 reduce/exercise1/Ccode/main.c|tail -15 

  printf("\nComputing Visited sites....     ");
  for(i=0;i<nBlocks;i++){
    printf("\b\b\b\b%3.0f%%",100.0*(double)i/(double)nBlocks);
    fflush(NULL);
    zero(space,size);

    for(n=0;n<sizeBlock;n++){
       for(k=0;k<NDIM;k++){
          mark[k]=(int)ceil(abs(x[(i*sizeBlock+n)*NDIM+k]-min[k])); 
        }              
        space[mark[0]+mark[1]*size[0]+mark[2]*size[0]*size[1]]=1;
        ave_vol[n]+=sumSpace(space,size);
   }
  }
  


The trajectory of the particle is in the filename: random3Dt.xyz. The structure of the file is 

In [41]:
head -10 reduce/random3Dt.xyz

1000000   
144.97984	-55.087448
2.232421	-137.551697
168.748016	-0.000083
#time x y z 
1 0.0 0.0 0.0
2 0.036407981837455534 0.038226280215159406 0.03640427649996404
3 -0.06905383848196306 0.058088535372306946 0.05241198500306482
4 -0.11965267621336795 0.0550593544445928 -0.05898333989857282
5 -0.0942907350638626 0.2754845522401567 -0.06168650892253492


In the io.c file, you will find the functions to read and extract the positions of the file and they are already written at the beginning of the main.cu file. 


Use the template main.cu, and the files: kernels.cu, io.c to implement the exercise.

In [19]:
!cd reduce/exercise1

# Scan

The scan algorithm is one of the building blocks for many parallel algorithms. The __inclusive scan__ is defined as for  a given input array, it generates a new array where each element $j$ of the resulting array is the sum of all elements up to and including the $j$ of the input array. 


There are many ways of implementing this algorithm, we are going to start with the most simple to understand and easy to implement for practical matter.  

At first, it seems that we cannot separate those operations. To picture better algorithm, let's expand the result for an array of 8 elements, __b__ would be:

We can separate this sum by adding each element $i$ the previous $i-2^{i}$ elements, for elements bigger and equal than $2^{i}$ at different steps.  The C code of such implementation would be

Note. The number of steps went down to log(size)=log(8)=3, instead of 8.
Now, let's implement it within an example.

## Example 1

In this example, we are going to implement the scan operation in a program that generates an array of N elements with a given value. 

In [ ]:
scan/example1/main 10 1

If we generate an array of 20 elements with a value of 1. The output will be,

In [18]:
more scan/example1/output.dat

=1.000000
2.000000
3.000000
4.000000
5.000000
6.000000
7.000000
8.000000
9.000000
10.000000
11.000000
12.000000
13.000000
14.000000
15.000000
16.000000
17.000000
18.000000
19.000000
20.000000
>

We implement the __scan__ algorithm in two kernels called __scanA__ and __scanB__. The __scanA__ will copy the values that are not going to be change at any step to the resulting array, and the __scanB__ will add the values of the element $i-2^{i}$ to the $i$ element each $i$ step.   

In [24]:
head -39 scan/example1/kernerls.cu|tail -29

__global__ void scan_B(float *x,float *dx,unsigned long int size,int jump){

    int gId = (blockIdx.y*gridDim.x+blockIdx.x)*blockDim.x+threadIdx.x;
 
    float x0,x1;

       x0=x[gId];
       __syncthreads();

       x1=x[gId+jump];
       __syncthreads();


       if((gId+jump)<size)
        dx[gId+jump]=x1+x0;
}

__global__ void scan_A(float *x,float *dx,unsigned long int size){

    int gId = (blockIdx.y*gridDim.x+blockIdx.x)*blockDim.x+threadIdx.x;

    float x0;

       x0=x[gId];

      if(gId<size)
        dx[gId]=x0;

}


Now, let's implement it a more practical exercise.

## Exercise 1

In this exercise, we are going to implement a simple code that computes the time average of the second moment of a random variable $\langle x^{2}\rangle$. We will provide the entire C routine, and you will implement it in a template CUDA. 

## Computing the second moment of a random variable 


To make the implementation more significant, we are going to analyze the positions of a particle that moves in 3D randomly. Thus, the time average of the second moment is the variation in time of the mean square displacement of the particle __msd(t)__ which for a Brownian motion, __msd(t)__ is linearly proportional to __t__ .

For this example, we provide the trajectory of the particle, which is in file: random3Dt.xyz

First, we are going to explain how it would be in C code, then we will convert it in CUDA.

You will find all files in the scan/exercise1 directory 

In [18]:
head random3Dt.xyz

1000000   
#time x y z   
1 0.0 0.0 0.0
2 0.036407981837455534 0.038226280215159406 0.03640427649996404
3 -0.06905383848196306 0.058088535372306946 0.05241198500306482
4 -0.11965267621336795 0.0550593544445928 -0.05898333989857282
5 -0.0942907350638626 0.2754845522401567 -0.06168650892253492
6 -0.20169277539001323 0.44299126333386823 0.033296495974359915
7 -0.2257369555712272 0.5597075775431412 0.023790254634477363
8 -0.23419574273545854 0.6336599820864995 0.048494147655551274


To aid its implementation, we provide the C code that computes msd(t). In addition, we divide in blocks our trajectory. For each block, we are going to compute the distance of the particle between the starting time of the block to each time-step. Then, we average the blocks at each time.

The number of blocks (nbl) is going to be constant and is an input parameter, to simplify our example.

The C code has the following structure
 
    * Declaration of variables and functions
    * Memory Allocation
    * Initialization of variables
    * Core of the program (performing the actual computation)
    * Output file

We are going to keep the same structure for our CUDA code example to emphasize the differences between C and CUDA. 

__Declaration of variables and functions__ are at the beginning of the program. To see them, we can type:

In [31]:

head -24 scan/exercise1/Ccode/main.c|tail -16

int read_initial_parameters ( char filename[20],long int *nconf);
int read_file( char filename[20],long int nconf, float *x,int *time);
void write_output(char fname[MAX], long int nconf, float *msd, int *time,int size);

int main(int argc, char *argv[])
{
  float *x, *msd, rtot[NDIM], r2;
  int nbl,sizebl,*time;
  long int nconf,t;
  FILE *fp;
  int i,k;
  clock_t start, diff;

 if (argc < 3){
    printf("Usage: msd_c <pos file> <nbl> \n");
    return(0);


where,
__read_initial_parameters__ obtains the number steps of the particle. 
__read_file__  opens the position file.
__write_output__ save the msd as a function of time.

Now, we focus in the core part of the program, computing the msd, which uses the following variables,

To see __the core__ of the program, we can type 

In [32]:
head -70 scan/exercise1/Ccode/main.c|tail -15

  printf("\nCaculating MSD...     ");
  for (i=0; i<nbl; i++) {
      printf("\b\b\b\b%3.0f%%", 100.0*(float)i/(float)nbl);
      fflush(NULL);      
      r2=0;
      for (t=1; t<sizebl; t++) {
          rtot[0] = rtot[1] = rtot[2] = 0.0;
	  for (k=0; k<NDIM; k++){
	       rtot[k] += (x[(i*sizebl+t)*NDIM+k]-x[(i*sizebl+t-1)*NDIM+k]); 
               r2 += rtot[k]*rtot[k];
	  }  
          msd[t] += r2;
         
      }
  }


This double loop is what we need to parallelize. In order to do so, we need to break the double for-loop. Note that msd[t] is the addition of all previous msd values, a direct application of the __Scan__ algorithm.


In [42]:
cd scan/exercise1/Ccode

To compile the C code, we write

In [44]:
gcc main.c io.c -o main_c 

To run the code, we type

In [45]:
./main_c

Usage: msd_c <pos file> <nbl> 


If you do not specify any parameters, the code will tell you what information you need to provide, where nbl is the size of the block, thus 

In [55]:
cd ..

In addition, we provide the templates for main.cu kernel.cu and io.c files as previous exercises in scan/exercise1/

# Histograms

The final exercise is to compute a histogram. A histogram counts the occurence of a value within a set of numbers. Only using scan and reduce, we can easily compute a histogram. 

Let's compute the histogram of the values generated in __exercise 2B__ from the __Basics__ section.

There are many ways of computing histograms; a easy implementation is to reduce only the values that belong to each bin identifier. This method is called __reduce by key__. Our key is going to be the number of the bin that each value belongs. We first need to generate the keys for all values, then we reduce the ones that have the same value.

Let's take a look of the generation of the key kernel,

In [59]:
head -23 histo/example/kernerls.cu |tail -14

__global__ void keyGenerator(float *d_in, int *bin, float binSize,unsigned long int size){
    int index_x = blockIdx.x * blockDim.x + threadIdx.x;
    int index_y = blockIdx.y * blockDim.y + threadIdx.y;
    int grid_width = gridDim.x * blockDim.x;
    int gId  = index_y * grid_width + index_x;
    int key;
    float local;

    if(gId<size){
     local=d_in[gId];
     key = (int) (local/binSize);
     bin[gId]=key;
    }
}


And the __reducebykey__ kernel which has a small variation of the reduce kernel. We do not care about the value of the element to reduce, but the key value. If it is the same key that we are reducing, then we add 1 otherwise nothing.

In [61]:
head -66 histo/example/kernerls.cu |tail -43

__global__ void reducebykey (int *bin, float *d_out,unsigned long int size,int key){

    extern __shared__ float sdata[];

    int index_x = blockIdx.x * blockDim.x + threadIdx.x;
    int index_y = blockIdx.y * blockDim.y + threadIdx.y;
    int grid_width = gridDim.x * blockDim.x;

    // map the two 2D indices to a single linear, 1D index
    int gId  = index_y * grid_width + index_x;
    int lId = threadIdx.x;
    int bId  = blockIdx.y * gridDim.x + blockIdx.x;
    float filter;

    int lkey = bin[gId];

    if (lkey == key)
       filter=1.0;
    else
      filter=0.0;
   
   if(gId<size){
     sdata[lId]=filter;
   }
   else
    sdata[lId]=0.0;

    __syncthreads();    

   for(unsigned int s =blockDim.x/2;s>0;s >>=1){

        if(lId<s){
	 sdata[lId]+=sdata[lId+s];
        }
    __syncthreads();

   }
    if(lId==0){
      d_out[bId]=sdata[0];
    }     

}
__global__ void reduce (float *d_in,float *d_out,unsigned long int size){


Now, let's practice in an example.

Compute the histogram of the trajectory file random3Dt.xyz. 
Fill in the blanks of the main.cu and use the kernels.cu file form the example above. 
All files are located in __histo/exercise/__.